In [306]:
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager   # 크롬드라이버 동작이 잘 안되서 파이어폭스로 실행
# options = webdriver.ChromeOptions()
options = webdriver.FirefoxOptions()

# 다운로드 경로 설정
default_download_path = "download_dir"

# Firefox 프로파일 설정
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", default_download_path)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream")

# 웹 드라이버 설정
driver = webdriver.Firefox(options=options)

In [307]:
google_trends_url = "https://trends.google.co.kr/trends?geo=KR&hl=ko"
driver.get(google_trends_url)

In [308]:
from selenium.webdriver.common.by import By

# 탐색 버튼 클릭
driver.find_elements(By.CLASS_NAME, "VfPpkd-RLmnJb")[1].click()
driver.implicitly_wait(5)

In [380]:
# 브라우저 줌 레벨을 50%로 설정
driver.execute_script("document.body.style.zoom='50%'")

In [381]:
# 검색 키워드 목록
keyword_list = ["알레르기성 결막염", "결막염", "알레르기 질환", "알레르기", "수족구병", "수족구", "온열질환", "장염", "식중독", "폐렴", "호흡계통의 질환", "감기", "인플루엔자", "한랭질환"]

In [382]:
from selenium.webdriver.common.keys import Keys
import time

def input_text(input_element, text):
    # input_element.send_keys(Keys.CONTROL + "a")
    # input_element.send_keys(Keys.COMMAND + "a")
    input_element.clear()
    input_element.send_keys(text)
    time.sleep(0.2)
    input_element.send_keys(Keys.ENTER)

In [383]:
import pandas as pd

date_format = "%-m/%-d/%Y"
date_format_text = "%m%d%Y"

# 시작 날짜와 종료 날짜에 대해 월별로 첫날과 마지막 날을 생성
# 각 달의 첫날과 마지막 날을 문자열로 추가
def date_range(start, end):
    date_range = pd.date_range(start=start, end=end, freq='MS')  # 각 달의 시작일
    search_date_list = []
    text_date_list = []
    for date in date_range:
        first_day = date.strftime(date_format)
        last_day = (date + pd.offsets.MonthEnd(0)).strftime(date_format)
        search_date_list.extend([first_day, last_day])
        
        first_day_text = date.strftime(date_format_text)
        last_day_text = (date + pd.offsets.MonthEnd(0)).strftime(date_format_text)
        text_date_list.extend([first_day_text, last_day_text])
    
    return search_date_list, text_date_list

search_date_list, text_date_list = date_range("20190101", "20240501")
print("Search Date List:", search_date_list)
print("Text Date List:", text_date_list)

Search Date List: ['1/1/2019', '1/31/2019', '2/1/2019', '2/28/2019', '3/1/2019', '3/31/2019', '4/1/2019', '4/30/2019', '5/1/2019', '5/31/2019', '6/1/2019', '6/30/2019', '7/1/2019', '7/31/2019', '8/1/2019', '8/31/2019', '9/1/2019', '9/30/2019', '10/1/2019', '10/31/2019', '11/1/2019', '11/30/2019', '12/1/2019', '12/31/2019', '1/1/2020', '1/31/2020', '2/1/2020', '2/29/2020', '3/1/2020', '3/31/2020', '4/1/2020', '4/30/2020', '5/1/2020', '5/31/2020', '6/1/2020', '6/30/2020', '7/1/2020', '7/31/2020', '8/1/2020', '8/31/2020', '9/1/2020', '9/30/2020', '10/1/2020', '10/31/2020', '11/1/2020', '11/30/2020', '12/1/2020', '12/31/2020', '1/1/2021', '1/31/2021', '2/1/2021', '2/28/2021', '3/1/2021', '3/31/2021', '4/1/2021', '4/30/2021', '5/1/2021', '5/31/2021', '6/1/2021', '6/30/2021', '7/1/2021', '7/31/2021', '8/1/2021', '8/31/2021', '9/1/2021', '9/30/2021', '10/1/2021', '10/31/2021', '11/1/2021', '11/30/2021', '12/1/2021', '12/31/2021', '1/1/2022', '1/31/2022', '2/1/2022', '2/28/2022', '3/1/2022', '

In [384]:
import time
import os
    
def get_categories():
    categories_div = driver.find_element(By.ID, "compare-pickers-wrapper")
    categories = categories_div.find_elements(By.CLASS_NAME, "compare-picker")
    return categories

def select_date_category(date_categorie):
    date_categorie.click()  # 날짜 범위 선택
    time.sleep(0.5)
    option_list = driver.find_elements(By.TAG_NAME, "md-option")
    option_list[-1].click()  # 맞춤 날짜
    time.sleep(1)

def get_date_input():
    date_input = driver.find_elements(By.CLASS_NAME, "md-datepicker-input")
    start_input = date_input[0]
    end_input = date_input[1]
    return start_input, end_input

# 날짜 범위 지정
def input_date(start_input, end_input, start_date, end_date):
    input_text(end_input, end_date)
    input_text(start_input, start_date)
    time.sleep(1)

    driver.find_elements(By.CLASS_NAME, "custom-date-picker-dialog-button")[1].click()
    time.sleep(0.5)

def set_date_range(date_categorie, start_date, end_date):
    select_date_category(date_categorie)
    time.sleep(1)
    start_input, end_input = get_date_input()
    time.sleep(1)
    input_date(start_input, end_input, start_date, end_date)

# 웹 검색: ""
# 뉴스 검색: "news"
# 이미지 검색: "images"
# Google 쇼핑: "froogle"
# 유튜브 검색: "youtube"
def select_search_type(search_type_categorie, search_type="web"):
    search_type_categorie.click()  # 검색 유형 선택
    time.sleep(1)

    if search_type == "web":
        search_type = ""  # web의 value는 ""

    element = driver.find_element(By.CSS_SELECTOR, f'md-option[value="{search_type}"]')
    element.click()
    time.sleep(0.5)
    
def input_keyword(keyword):
    search_keyword_input = driver.find_element(By.CSS_SELECTOR, "input.ng-valid.ng-valid-required.flex")
    input_text(search_keyword_input, keyword)  # 검색어 입력
    time.sleep(0.5)

def get_download_button():
    csv_download_button_list = driver.find_elements(By.CLASS_NAME, "widget-actions-item.export")
    interest_over_time_download = csv_download_button_list[0]  # 시간 흐름에 따른 관심도 변화
    interest_by_subregion_download = csv_download_button_list[1]  # 하위 지역별 관심도
    # related_topics_download = csv_download_button_list[2]  # 관련 주제
    # related_queries_download = csv_download_button_list[3]  # 관련 검색어
    return interest_over_time_download, interest_by_subregion_download

multiTimeline_path = os.path.join(default_download_path, "multiTimeline.csv")
geoMap_path = os.path.join(default_download_path, "geoMap.csv")

def rename_downloaded_files(new_multiTimeline_path, new_geoMap_path):
    os.rename(multiTimeline_path, new_multiTimeline_path)
    os.rename(geoMap_path, new_geoMap_path)
        

def download_csv_files(move_path, new_multiTimeline_filename, new_geoMap_filename):
    time.sleep(1)
    interest_over_time_download, interest_by_subregion_download = get_download_button()
    
    interest_over_time_download.click()
    time.sleep(1)
    interest_by_subregion_download.click()
    time.sleep(1)

    new_multiTimeline_filename = f"{new_multiTimeline_filename}_multiTimeline.csv"
    new_geoMap_filename = f"{new_geoMap_filename}_geoMap.csv"
    new_multiTimeline_path = os.path.join(move_path, new_multiTimeline_filename)
    new_geoMap_path = os.path.join(move_path, new_geoMap_filename)
    rename_downloaded_files(new_multiTimeline_path, new_geoMap_path)

def download_csv_by_search_type(search_type_categorie, search_type, move_path, new_multiTimeline, new_geoMap):
    select_search_type(search_type_categorie, search_type)
    time.sleep(1)

    new_multiTimeline_filename = f"{new_multiTimeline}_{search_type}_multiTimeline.csv"
    new_geoMap_filename = f"{new_geoMap}_{search_type}_geoMap.csv"
    download_csv_files(move_path, new_multiTimeline_filename, new_geoMap_filename)
    driver.implicitly_wait(3)

In [385]:
default_download_path = "download_dir"

def search_keyword(keyword):
    move_path = default_download_path + "/" + keyword
    os.mkdir(move_path)

    for index in range(0, len(search_date_list), 2):
        input_keyword(keyword)

        start_date = search_date_list[index]
        end_date = search_date_list[index+1]

        categories = get_categories()
        set_date_range(categories[1], start_date, end_date)
        
        format_start_date = text_date_list[index]
        format_end_date = text_date_list[index+1]
        new_multiTimeline = f"{keyword}_{format_start_date}_{format_end_date}"
        new_geoMap = f"{keyword}_{format_start_date}_{format_end_date}"
        download_csv_files(move_path, new_multiTimeline, new_geoMap)

        # search_type_categorie = categories[-1]
        # download_csv_by_search_type(search_type_categorie, "web", move_path, new_multiTimeline, new_geoMap)
        # download_csv_by_search_type(search_type_categorie, "youtube", move_path, new_multiTimeline, new_geoMap)
        # print(f"{format_start_date} - {format_end_date} 진행완료")

In [388]:
for keyword in keyword_list:
    print(f"{keyword} 검색 중...")
    search_keyword(keyword)
    print(f"{keyword} 검색 완료")

알레르기성 결막염 검색 중...


IndexError: list index out of range